In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import HGate, CXGate, QFT
from qiskit.circuit import Parameter, ParameterVector, Barrier
from qiskit.circuit import Gate
import numpy as np

from qiskit_algorithms.minimum_eigensolvers.vqe import VQE
from qiskit_algorithms.optimizers import SPSA

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

In [2]:
num_q_orbital=4
num_q_site=4

qc_total=QuantumCircuit(num_q_orbital+num_q_site)

qc_orbital=QuantumCircuit(num_q_orbital) # orbital register

qc_site=QuantumCircuit(num_q_site) # site register
qc_p=QuantumCircuit(num_q_site) # minimum unit for representing quantum number p
qc_QFT=QuantumCircuit(num_q_site) # quantum circuit for executing quantum fourier transform

optimizer = SPSA()

In [3]:
#################
# Site register #
#################

def decimal_to_binary(num_q_site, p):
    bin_array=list()
    denominator=p
    for i in reversed(range(num_q_site)):
        bit=denominator//(2**i)
        bin_array.append(bit)
        denominator=denominator-bit*(2**i)
    return bin_array

def QuantumFourierTransform(num_q_site):
    qc_QFT=QuantumCircuit(num_q_site, name='QFT')
    
    # Before SWAP
    for i in range(num_q_site):        
        qc_QFT.h(i)
        for j in range(num_q_site-1-i):
            theta=(2*np.pi)/(2**(j+2))
            qc_QFT.crz(theta=theta , control_qubit=j+1+i , target_qubit=i)
        qc_QFT.barrier()
    
    # after SWAP
    for i in range(num_q_site//2):
        qc_QFT.swap(i, num_q_site-1-i)
    
    return qc_QFT

def ansatze_site(num_q_site, p):
    bin_array=decimal_to_binary(num_q_site, p)
    qc_ansatze_site = QuantumCircuit(num_q_site, name=f'site_register[p={p}]')
    
    for i, bit in enumerate(bin_array):
        if bit==1:
            qc_ansatze_site.x(i)
    qc_ansatze_site.barrier()
    
    qc_tmp=QuantumFourierTransform(num_q_site)
    inst=qc_tmp.to_instruction()
    qc_ansatze_site.append(inst, range(num_q_site))
    
    # qc_ansatze_site.append(QFT(num_q_site, name=f'QFT[p={p}]'), range(num_q_site))
    
    return qc_ansatze_site

In [4]:
####################
# Orbital register #
####################

def A(theta, phi):
    # minimum unit of the number preserving ansatze ; included into the qc_orbital
    qc_ansatze_unit=QuantumCircuit(2, name="A") 
    qc_ansatze_unit.cx(0,1)
    qc_ansatze_unit.rz(phi+np.pi, 0).inverse()
    qc_ansatze_unit.ry(theta+np.pi/2, 0).inverse()
    qc_ansatze_unit.cx(1,0)
    qc_ansatze_unit.ry(theta+np.pi/2, 0)
    qc_ansatze_unit.rz(phi+np.pi, 0)
    qc_ansatze_unit.cx(0,1)
    return qc_ansatze_unit

def ansatze_orbital_encoding(energy_level):
    qc_ansatze_o_ecdg=QuantumCircuit(num_q_orbital)
    qc_ansatze_o_ecdg.x(energy_level)
    return qc_ansatze_o_ecdg    

def ansatze_level(energy_level, params_level):
    qc_ansatze_E_level=QuantumCircuit(num_q_orbital, name='level_'+str(energy_level))
    for i in range(num_q_orbital-energy_level-1):
        inst=A(theta=params_level[2*i], phi=params_level[2*i+1]).to_instruction()
        qc_ansatze_E_level.append(inst, [i+energy_level, i+energy_level+1])
    return qc_ansatze_E_level

In [5]:
#########################
# Hamiltonian Execution #
#########################

def list_to_str(list_data):
    str_first = ''
    for str_data in list_data:
        str_first += str_data
    return str_first

########### 여기 줄 잘 바꾸샘
def custom_hamiltonian(qubit_number):
    list_data = []
    for i in range(qubit_number):
        raw_data = ['I']*qubit_number
        raw_data[i] = 'Z'
        data = list_to_str(raw_data)
        list_data.append(data)
        
    return SparsePauliOp(list_data,coeffs=[0.1*i for i in range(qubit_number)])


def vqe(num_q_orbital, num_q_site, energy_level, previous_params_list):
    # divide quantum circuit into orbital register part and site register part each
    circuit_divider={'orbital':[], 'site':[]}
    for i in range(num_q_orbital+num_q_site):   
        if i<num_q_orbital:
            circuit_divider['orbital'].append(i)
        else:
            circuit_divider['site'].append(i)
    
    if previous_params_list==None: # target energy level is 0th : ground state band
        qc_0th=QuantumCircuit(num_q_orbital+num_q_site, name='ansatze_0th_level')
        
        # orbital ansatze for 0th level
        qc_orbital_0th=ansatze_orbital_encoding(energy_level) # orbital encoding for 0th energy level
        num_params=2*(num_q_orbital-energy_level-1)
        params_0th=ParameterVector("θ", num_params)
        qc_tmp=ansatze_level(energy_level, params_level=params_0th)
        inst=qc_tmp.to_instruction()
        qc_orbital_0th.append(inst, range(num_q_orbital))
        
        # parameter list : 2 dimensional list [columns : parameter for each parametric gate, rows : p's(0,1,...,2**n-1)]
        params_fixed_0th=list()
        
        # eigen energy list : 1 dimensional list [row : eigen energy for sequential p's(0,1,...,2**n-1)]
        eigen_energy_0th=list()
        
        # entire ansatze for 0th level and optimization with the for loop containing site ansatze and its corresponding quantum number p
        for p in range(2**num_q_site):
            # site ansatze for 0th level
            qc_site_0th=ansatze_site(num_q_site, p)
            
            # construct entire ansatze
            inst_orbital=qc_orbital_0th.to_instruction()
            inst_site=qc_site_0th.to_instruction()
            qc_0th.append(inst_orbital, circuit_divider['orbital'])
            qc_0th.append(inst_site, circuit_divider['site'])
        
            def loss(x):
                estimator = Estimator()
                pub = (qc_0th,custom_hamiltonian(4),x)
                job = estimator.run([pub])
                result = job.result()[0]
                return -np.sum(result.data.evs)

            result = optimizer.minimize(loss, x0 = [0.2]*qc_0th.num_parameters)
            print(f'0th eigen energy information (p={p}) : {result}')
            params_fixed_0th.append(result.x) # further obtained
            eigen_energy_0th.append(result.fun)
        
        return qc_orbital_0th, params_fixed_0th, eigen_energy_0th
    
    else: # set ansatze for nonzero energy level 
        qc_ith=QuantumCircuit(num_q_orbital+num_q_site, name=f'ansatze_{i}th_level')
        
        # current energy level
        qc_orbital_ith=ansatze_orbital_encoding(energy_level)    
        num_params=2*(num_q_orbital-energy_level-1)
        params=ParameterVector("θ", num_params)
        qc_tmp=ansatze_level(energy_level, params_level=params)
        inst_current=qc_tmp.to_instruction()
        qc_orbital_ith.append(inst_current, range(num_q_orbital))
        
        # parameter list : 2 dimensional list [columns : parameter for each parametric gate, rows : p's(0,1,...,2**n-1)]
        params_fixed_ith=list()
        
        # eigen energy list : 1 dimensional list [row : eigen energy for sequential p's(0,1,...,2**n-1)]
        eigen_energy_ith=list()
        
        # entire ansatze for ith level with other fixed ansatze's and optimization with the for loop containing site ansatze and its corresponding quantum number p
        for p in range(2**num_q_site-1):
            # previous energy level with fixed parameters
            for i in reversed(range(energy_level)):
                qc_orbital_ith.barrier()
                params=previous_params_list[i][p]
                previous_level_qc=ansatze_level(energy_level=i, params_level=params)
                inst=previous_level_qc.to_instruction()
                qc_orbital_ith.append(inst, range(num_q_orbital))
                
            # site ansatze for ith level
            qc_site_ith=ansatze_site(num_q_site, p)
            
            # construct entire ansatze
            inst_orbital=qc_orbital_ith.to_instruction()
            inst_site=qc_site_ith.to_instruction()
            qc_ith.append(inst_orbital, circuit_divider['orbital'])
            qc_ith.append(inst_site, circuit_divider['site'])
            
            def loss(x):
                estimator = Estimator()
                pub = (qc_ith,custom_hamiltonian(4),x)
                job = estimator.run([pub])
                result = job.result()[0]
                return -np.sum(result.data.evs)

            result = optimizer.minimize(loss, x0 = [0.2]*qc_ith.num_parameters)
            print(f'{energy_level+1}th eigen energy information : {result}')
            params_fixed_ith.append(result.x) # further obtained
            eigen_energy_ith.append(result.fun)
        
        return qc_orbital_ith, params_fixed_ith, eigen_energy_ith

In [6]:
upper_bound_level=2
params_library = dict()
eigen_energy_library = dict()

for i in range(upper_bound_level+1):
    if i==0:
        _ , params_fixed_0th, eigen_energy_0th = vqe(
            num_q_orbital=num_q_orbital,
            num_q_site=num_q_site,
            energy_level=i, 
            previous_params_list=None
        )
        params_library['level_'+str(i)] = params_fixed_0th
        eigen_energy_library['lelvel_'+str(i)]=eigen_energy_0th
    else:
        params_send = [params_library['level_'+str(j)] for j in range(i)]
        _ , params_fixed_ith, eigen_energy_ith = vqe(
            num_q_orbital=num_q_orbital,
            num_q_site=num_q_site,
            energy_level=i, 
            previous_params_list=params_send
        )
        params_library['level_'+str(i)] = params_fixed_ith
        eigen_energy_library['lelvel_'+str(i)]=eigen_energy_ith

ValueError: The number of qubits of the circuit (8) does not match the number of qubits of the ()-th observable (4).